[ML] Bosch Production Line Performance
- 문제 정의: Bosch사의 조립 공정 라인의 모든 단계에 대한 데이터를 분석하여,
제품의 불량을 예측

#### 기본적인 import

In [17]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tqdm
import gc
import sys
import warnings
warnings.filterwarnings("ignore")

In [18]:
from scipy import stats

In [19]:
cd /content/drive/MyDrive/Colab Notebooks/2024_Kaggle/Bosch_Production_Line_Project/

/content/drive/MyDrive/Colab Notebooks/2024_Kaggle/Bosch_Production_Line_Project


### 데이터 전처리
1. 데이터의 형태를 확인한다 (info, isna.sum()등)

2. 데이터의 특징을 파악한다 (변수명이 L-S-F의 결합이다/이 데이터에서의 결측치는 추정해서 채워야하는 값이 아니다)

3. 가장 중요한 특징이 될 변수를 선정하고 해당 변수를 기준으로 정리한다

    (1) station 기준으로 숫자형 변수 : 컬럼을 s값으로 변경하고 정리한다

    (2) station을 기준으로 범주형 변수 : 각 station별 발생가능 code를 True, False로 정리한다

#### 데이터 확인

In [20]:
train_date = pd.read_csv('data/train_date.csv.zip', nrows=10000)
train_numeric = pd.read_csv('data/train_numeric.csv.zip', nrows=10000)
train_category = pd.read_csv('data/train_categorical.csv.zip', nrows=10000)
test_data = pd.read_csv('data/test_date.csv.zip', nrows=10000)
test_numeric = pd.read_csv('data/test_numeric.csv.zip', nrows=10000)
test_category = pd.read_csv('data/test_categorical.csv.zip', nrows=10000)

- 데이터 특징 정리
 - numeric/categorical data로 분류
 - 데이터샘플은 적고 특징(컬럼)이 굉장히 많은 데이터로써 특징 추출이 매우 중요한 문제
 - 결측치가 매우 많다.
 - 비식별화된 특징이 매우 많다
 - 불량 예측문제 답게 클래스 불균형 문제가 심각
 - 독립된 ID별 - L(제조 라인)_S(제조 스테이션)_F(기능 번호)
 - ID 100이 10번 제조 라인에서 생산될 시, 다른 제조 라인은 모두 결측치가 될 수 밖에 없는 구조

In [21]:
display(train_date.head(3), train_date.columns, test_data.head(3), test_data.columns)

,Id,L0_S0_D1,L0_S0_D3,L0_S0_D5,L0_S0_D7,L0_S0_D9,L0_S0_D11,L0_S0_D13,L0_S0_D15,L0_S0_D17,...,L3_S50_D4246,L3_S50_D4248,L3_S50_D4250,L3_S50_D4252,L3_S50_D4254,L3_S51_D4255,L3_S51_D4257,L3_S51_D4259,L3_S51_D4261,L3_S51_D4263
0,4,82.24,82.24,82.24,82.24,82.24,82.24,82.24,82.24,82.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,1618.70,1618.70,1618.70,1618.70,1618.70,1618.70,1618.70,1618.70,1618.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Index(['Id', 'L0_S0_D1', 'L0_S0_D3', 'L0_S0_D5', 'L0_S0_D7', 'L0_S0_D9',
       'L0_S0_D11', 'L0_S0_D13', 'L0_S0_D15', 'L0_S0_D17',
       ...
       'L3_S50_D4246', 'L3_S50_D4248', 'L3_S50_D4250', 'L3_S50_D4252',
       'L3_S50_D4254', 'L3_S51_D4255', 'L3_S51_D4257', 'L3_S51_D4259',
       'L3_S51_D4261', 'L3_S51_D4263'],
      dtype='object', length=1157)

,Id,L0_S0_D1,L0_S0_D3,L0_S0_D5,L0_S0_D7,L0_S0_D9,L0_S0_D11,L0_S0_D13,L0_S0_D15,L0_S0_D17,...,L3_S50_D4246,L3_S50_D4248,L3_S50_D4250,L3_S50_D4252,L3_S50_D4254,L3_S51_D4255,L3_S51_D4257,L3_S51_D4259,L3_S51_D4261,L3_S51_D4263
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Index(['Id', 'L0_S0_D1', 'L0_S0_D3', 'L0_S0_D5', 'L0_S0_D7', 'L0_S0_D9',
       'L0_S0_D11', 'L0_S0_D13', 'L0_S0_D15', 'L0_S0_D17',
       ...
       'L3_S50_D4246', 'L3_S50_D4248', 'L3_S50_D4250', 'L3_S50_D4252',
       'L3_S50_D4254', 'L3_S51_D4255', 'L3_S51_D4257', 'L3_S51_D4259',
       'L3_S51_D4261', 'L3_S51_D4263'],
      dtype='object', length=1157)

- Numeric data
 - L-S-F 조합, 969개의 컬럼명 존재
 - L(생산라인), S(station), F(feature)
 - L과 S의 값 개수와 특징 살피기
- Category data  
 - 2140개의 컬럼 존재
- 범주형, 숫자형의 컬럼명 모두 불일치

In [22]:
display(train_numeric.head(3), train_category.head(3)) #, test_numeric.head(3), test_category.head(3))

,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,...,L3_S50_F4245,L3_S50_F4247,L3_S50_F4249,L3_S50_F4251,L3_S50_F4253,L3_S51_F4256,L3_S51_F4258,L3_S51_F4260,L3_S51_F4262,Response
0,4,0.030,-0.034,-0.197,-0.179,0.118,0.116,-0.015,-0.032,0.020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,7,0.088,0.086,0.003,-0.052,0.161,0.025,-0.015,-0.072,-0.225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


,Id,L0_S1_F25,L0_S1_F27,L0_S1_F29,L0_S1_F31,L0_S2_F33,L0_S2_F35,L0_S2_F37,L0_S2_F39,L0_S2_F41,...,L3_S49_F4225,L3_S49_F4227,L3_S49_F4229,L3_S49_F4230,L3_S49_F4232,L3_S49_F4234,L3_S49_F4235,L3_S49_F4237,L3_S49_F4239,L3_S49_F4240
0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# 두 데이터의 컬럼명 일치 여부 확인

# 컬럼 추출
numeric_columns = train_numeric.columns
category_columns =  train_category.columns
print(len(numeric_columns), len(category_columns))

# 컬럼명 일치 여부 확인
columns_match = numeric_columns.equals(category_columns)

if columns_match: # 컬럼명 일치한다면 true
  print("true")
else:
  print("false")

# 추가: 일치하지 않는 컬럼명 출력
numeric_only = set(numeric_columns) - set(category_columns)
category_only = set(category_columns) - set(numeric_columns)
print(len(numeric_only), len(category_only)) # id 제외하고 모두 불일치

970 2141
false
969 2140


#### numeric data 탐색
- 변수 사이의 관계를 살펴보고, 데이터가 어떤 형태로 어떤 특징으로 분포하는지 확인
- (어떤식으로 전처리를 해야 불량품을 예측하는 것에 효율적일지 고민하면서 탐색)

목표 변수와 독립변수 분리해서 전처리 진행

In [24]:
df = train_numeric.copy()
df.set_index('Id', inplace=True) # id를 index로 설정

In [25]:
x = df.drop('Response', axis = 1) # 라벨 컬럼 분리
y = df['Response']

L(생산라인), S(스테이션) 수 확인

In [26]:
pd.Series(x.columns).str.split('_', expand=True)

,0,1,2
0,L0,S0,F0
1,L0,S0,F2
2,L0,S0,F4
3,L0,S0,F6
4,L0,S0,F8
...,...,...,...
963,L3,S50,F4253
964,L3,S51,F4256
965,L3,S51,F4258
966,L3,S51,F4260


총 L0, L1, L2, L3 라인 존재, 각 라인별 스테이션과 특징들을 담을 수 있도록 dict() 생성

In [27]:
# dict() 생성
line_station_feature_dict = dict()
# 라인별 정보
line_station_feature_dict['L0'] = {'station':[], 'feature':[]}
line_station_feature_dict['L1'] = {'station':[], 'feature':[]}
line_station_feature_dict['L2'] = {'station':[], 'feature':[]}
line_station_feature_dict['L3'] = {'station':[], 'feature':[]}

In [28]:
for col in x.columns:
  line, station, feature = col.split('_') # 언더바를 기준으로 분리
  if station not in line_station_feature_dict[line]['station']:
    line_station_feature_dict[line]['station'].append(station)
  if feature not in line_station_feature_dict[line]['feature']:
    line_station_feature_dict[line]['feature'].append(feature)

라인별 스테이션과 특징들 확인

In [29]:
# 라인별 스테이션
for line in line_station_feature_dict.keys():
  print(line_station_feature_dict[line]['station'])

['S0', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20', 'S21', 'S22', 'S23']
['S24', 'S25']
['S26', 'S27', 'S28']
['S29', 'S30', 'S31', 'S32', 'S33', 'S34', 'S35', 'S36', 'S37', 'S38', 'S39', 'S40', 'S41', 'S43', 'S44', 'S45', 'S47', 'S48', 'S49', 'S50', 'S51']


In [30]:
# 라인별 feature 확인
for line in line_station_feature_dict.keys():
  print(line_station_feature_dict[line]['feature'])

['F0', 'F2', 'F4', 'F6', 'F8', 'F10', 'F12', 'F14', 'F16', 'F18', 'F20', 'F22', 'F24', 'F28', 'F32', 'F36', 'F40', 'F44', 'F48', 'F52', 'F56', 'F60', 'F64', 'F68', 'F72', 'F76', 'F80', 'F84', 'F88', 'F92', 'F96', 'F100', 'F104', 'F109', 'F114', 'F116', 'F118', 'F122', 'F132', 'F136', 'F138', 'F142', 'F144', 'F146', 'F149', 'F155', 'F160', 'F165', 'F170', 'F175', 'F180', 'F185', 'F190', 'F195', 'F200', 'F205', 'F210', 'F219', 'F224', 'F229', 'F234', 'F239', 'F244', 'F249', 'F254', 'F259', 'F264', 'F269', 'F274', 'F282', 'F286', 'F290', 'F294', 'F298', 'F302', 'F306', 'F310', 'F314', 'F318', 'F322', 'F326', 'F330', 'F332', 'F334', 'F336', 'F338', 'F340', 'F342', 'F344', 'F346', 'F348', 'F350', 'F352', 'F354', 'F356', 'F358', 'F362', 'F366', 'F370', 'F374', 'F378', 'F382', 'F386', 'F390', 'F394', 'F397', 'F400', 'F403', 'F406', 'F409', 'F412', 'F415', 'F418', 'F421', 'F426', 'F431', 'F433', 'F435', 'F439', 'F449', 'F453', 'F455', 'F459', 'F461', 'F463', 'F466', 'F472', 'F477', 'F482', 'F4

In [31]:
# 데이터프레임으로 딕셔너리 정리
pd.DataFrame.from_dict(line_station_feature_dict)

,L0,L1,L2,L3
station,"[S0, S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, ...","[S24, S25]","[S26, S27, S28]","[S29, S30, S31, S32, S33, S34, S35, S36, S37, ..."
feature,"[F0, F2, F4, F6, F8, F10, F12, F14, F16, F18, ...","[F679, F683, F687, F691, F700, F719, F728, F73...","[F3036, F3040, F3047, F3051, F3055, F3062, F30...","[F3315, F3318, F3321, F3324, F3327, F3330, F33..."


- 스테이션: 중복값이 없음. 라인이 다르면 다른 종류의 스테이션이 나옴
- feature는 너무 많음
- 라인은 수가 너무 적음 (4개)
- 스테이션을 중심으로 데이터 전처리 진행


#### 데이터 전처리
- s를 기준으로 전처리해야함 (해당 스테이션을 지나가나 안지나가나를 True, False형태로 만들어서 변수추가)

- 모든 s의 유니크한 값을 변수로 두고 각 id당 1과 0으로 이뤄진 테이블 생성

In [32]:
# 모든 라인의 스테이션 전체 만들기
whole_station = []
for line in ['L0','L1','L2','L3']:
  whole_station += line_station_feature_dict[line]['station']
print(whole_station) # 리스트로 추가

['S0', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20', 'S21', 'S22', 'S23', 'S24', 'S25', 'S26', 'S27', 'S28', 'S29', 'S30', 'S31', 'S32', 'S33', 'S34', 'S35', 'S36', 'S37', 'S38', 'S39', 'S40', 'S41', 'S43', 'S44', 'S45', 'S47', 'S48', 'S49', 'S50', 'S51']


In [33]:
# 해당 스테이션 유무 0 1로 표현
station_x = []
for ind, row in x.iterrows():
  if sum(row.notnull()) == 0: # 각 행에서 결측치가 아닌 값들 (true) 의 개수 계산 -> 한 행이 모두 결측치라면
    station_x.append(np.zeros(len(whole_station))) # whole_station의 길이와 같은 길이를 가지는 1차원 배열을 생성하고, 그 배열의 모든 원소를 0으로 채우기
  else:
    not_null_columns = x.columns[row.notnull()] # 결측치가 아닌 컬럼 뽑기
    # .iloc[:,1]: 두번째 컬럼 인덱스 선택 (s선택), 고유값만 남기기
    station = pd.Series(not_null_columns).str.split('_', expand = True).iloc[:,1].drop_duplicates().tolist() # expand=True는 분리된 부분을 각각의 컬럼으로 확장하여 데이터프레임 형태로 반환
    station_x.append(np.isin(whole_station, station)) # station에 있는 값들이 whole_station에도 있으면 true, 없으면 false 반환



In [34]:
station_x = pd.DataFrame(station_x, index = x.index, columns = whole_station) # True, False 형태로  append
station_x = station_x.astype(int) # 1,0으로 바꾸기
print(station_x)

       S0  S1  S2  S3  S4  S5  S6  S7  S8  S9  ...  S40  S41  S43  S44  S45  \
Id                                             ...                            
4       1   1   1   0   1   0   0   1   1   0  ...    0    0    0    0    0   
6       0   0   0   0   0   0   0   0   0   0  ...    0    0    0    0    0   
7       1   1   1   0   0   1   1   0   1   0  ...    0    0    0    0    0   
9       1   1   1   0   1   0   0   1   1   0  ...    0    0    0    0    0   
11      1   1   0   1   1   0   0   1   1   0  ...    0    0    0    0    0   
...    ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ...  ...  ...  ...  ...   
19912   0   0   0   0   0   0   0   0   0   0  ...    0    0    0    0    0   
19915   1   1   0   1   0   1   0   1   1   0  ...    0    0    0    0    0   
19917   1   1   1   0   0   1   0   1   1   1  ...    0    0    0    0    0   
19921   0   0   0   0   0   0   0   0   0   0  ...    0    0    0    0    0   
19923   1   1   0   1   1   0   1   0   1   0  ...  

#### 통계량 확인
- 순서대로 진행하는 공정에서 제품별 서로 다른 공정의 길이를 거치기에 **공통적인 데이터를 분류**해야 한다.
- 그래서 time series와 같은 시계열에서 통계량의 추출은 '길이가 다른 시계열'을 분류할 때 자주 사용이 된다.

- 이상치, 결측치를 처리하는 함수와 각 'Id'별 대표 통계량들을 반환하는 함수를 생성한다.
 - 1) 각 값(df에서 컬럼별 입력된 값)에서 이상치 제거
 - 2) mean, var, max, min, RMS, kurtosis(첨도) 6가지 항목의 값을 구해서 변수로 생성

이상치 처리 함수

In [35]:
# val: 이상치 처리할 데이터 배열, w: IQR(Interquartile Range)을 기준으로 이상치 결정 가중치 default = 1.5
def remove_outliers(val, w=1.5):
    Q1 = np.quantile(val, 0.25)# 1사분위, 25번째 백분위
    Q3 = np.quantile(val, 0.75) # 3사분위, 75번째 백분위
    IQR = Q3 - Q1 # 사분위 범위

    low_cond = Q1 - w * IQR < val # IQR의 하한 경계보다 val 데이터가 큰지 여부 판단 => 작으면 이상치
    high_cond = Q3 + w * IQR > val # IQR의 상한 경계보다 val 데이터가 큰지 여부 판단 => 크면 이상치
    total_cond = np.logical_and(low_cond, high_cond) # 입력값 두개를 다 만족할 때 True 도출 => True값들만 선택

    return val[total_cond] # 이상치가 아닌 데이터만 포함된 배열 반환

결측치 및 통계량 반환 함수: 평균, 분산, 최대, 최소, 첨도, 제곱평균제곱근

In [36]:
def extract_statistical_feature(val): # val : Id별
    if val.notnull().sum() == 0:
        return pd.Series([0] * 6)
    else:
        val = val.copy().dropna() # 결측치 저리
        val = remove_outliers(val) # 이상치 처리

        val_mean = val.mean() # 평균
        val_var = val.var() # 분산: 데이터의 변동성
        val_max = val.max() # 최대값
        val_min = val.min() # 최소값
        val_kurtosis = stats.kurtosis(val) # 첨도: 데이터 분포의 꼬리의 두꺼운 정도(높을수록 outliers가 많음), 양수/0/음수
        val_rms = np.sqrt(sum(val**2 / len(val))) # 제곱평균제곱근 : Id별 특징이나 경향을 나타내는 대표값 중 하나

    return pd.Series([val_mean, val_var, val_max, val_min, val_kurtosis, val_rms])

In [37]:
# 함수를 통한 전처리, 각 컬럼별로 적용
state_feature_x = x.apply(extract_statistical_feature, axis=1) # axis = 0:index 방향, 1:column 방향
# display(state_feature_x)
# columns 이름 변경
state_feature_x.rename({0:'mean', 1:'variance', 2:'max', 3:'min', 4:'kurtosis', 5:'RMS'}, axis=1, inplace=True) # axis = 0:index, 1:column
state_feature_x
# 4,11 이 데이터 변동이 상대적으로 큼
# max-min: 데이터의 범위
# 7: 양의 첨도 - 극단값이 다른 ID들에 비해 많음
# RMS: 값이 높을수록 데이터 값이 큼 => 4

,mean,variance,max,min,kurtosis,RMS
Id,,,,,,
4,-0.013346,0.003363,0.118,-0.164,0.415134,0.059297
6,-0.002573,0.001208,0.096,-0.088,0.632356,0.034741
7,0.001774,0.000579,0.064,-0.060,1.093244,0.024048
9,-0.000248,0.000928,0.079,-0.084,0.837443,0.030357
11,-0.014000,0.003128,0.118,-0.157,0.315412,0.057489
...,...,...,...,...,...,...
19912,0.016471,0.002860,0.144,-0.115,0.126258,0.055807
19915,0.002671,0.002858,0.134,-0.140,0.808986,0.053348
19917,-0.002669,0.001658,0.108,-0.105,1.050556,0.040675


이제 'Id'별 공정 데이터와 통계량 데이터를 합쳐서 수치형 데이터로 통합하기

In [38]:
numeric_x = pd.merge(station_x, state_feature_x, left_index=True, right_index=True)

In [39]:
numeric_x.head(5)

,S0,S1,S2,S3,S4,S5,S6,S7,S8,S9,...,S48,S49,S50,S51,mean,variance,max,min,kurtosis,RMS
Id,,,,,,,,,,,,,,,,,,,,,
4,1,1,1,0,1,0,0,1,1,0,...,0,0,0,0,-0.013346,0.003363,0.118,-0.164,0.415134,0.059297
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,-0.002573,0.001208,0.096,-0.088,0.632356,0.034741
7,1,1,1,0,0,1,1,0,1,0,...,0,0,0,0,0.001774,0.000579,0.064,-0.060,1.093244,0.024048
9,1,1,1,0,1,0,0,1,1,0,...,0,0,0,0,-0.000248,0.000928,0.079,-0.084,0.837443,0.030357
11,1,1,0,1,1,0,0,1,1,0,...,0,0,0,0,-0.014000,0.003128,0.118,-0.157,0.315412,0.057489


#### 범주형 데이터

In [40]:
df_ca = train_category.copy()
df_ca.set_index('Id', inplace = True)

Id별 결측치가 아닌 값 확인

- numeric data보다 결측값이 더 많은 상태임
- id별로 어떤 값들이 있는지 확인하기
- 모든 데이터가 결측인 id도 존재

In [41]:
df_ca.iloc[0].dropna().unique()

array([], dtype=object)

In [42]:
df_ca.iloc[2].dropna().unique()

array(['T1'], dtype=object)

In [43]:
df_ca.iloc[50].dropna().unique()

array(['T1', 'T145'], dtype=object)

모든 행을 탐색 -> 결측값이 아닌 모든 코드값 저장

In [44]:
codes = []
for idx, row in df_ca.iterrows():
  for code in row.dropna().unique():
    if code not in codes:
      codes.append(code)
print(len(codes))

29


T코드가 등장한 스테이션 확인
- 모든 공정에서 나타나는 것이 아닌 특정 공정에서만 진행되었음

In [45]:
df_ca.iloc[2].dropna().index

Index(['L3_S29_F3317', 'L3_S29_F3320', 'L3_S29_F3323', 'L3_S29_F3326',
       'L3_S29_F3329', 'L3_S29_F3332', 'L3_S29_F3335', 'L3_S29_F3338',
       'L3_S29_F3341', 'L3_S29_F3344', 'L3_S29_F3347', 'L3_S29_F3350',
       'L3_S29_F3353', 'L3_S29_F3356', 'L3_S29_F3359', 'L3_S29_F3362',
       'L3_S29_F3364', 'L3_S29_F3366', 'L3_S29_F3369', 'L3_S29_F3372',
       'L3_S29_F3375', 'L3_S29_F3378', 'L3_S29_F3381', 'L3_S29_F3384',
       'L3_S29_F3387', 'L3_S29_F3390', 'L3_S29_F3392', 'L3_S29_F3394',
       'L3_S29_F3397', 'L3_S29_F3400', 'L3_S29_F3403', 'L3_S29_F3406',
       'L3_S29_F3409', 'L3_S29_F3411', 'L3_S29_F3414', 'L3_S29_F3416',
       'L3_S29_F3418', 'L3_S29_F3420', 'L3_S29_F3423', 'L3_S29_F3426',
       'L3_S29_F3429', 'L3_S29_F3432', 'L3_S29_F3435', 'L3_S29_F3438',
       'L3_S29_F3441', 'L3_S29_F3444', 'L3_S29_F3446', 'L3_S29_F3448',
       'L3_S29_F3451', 'L3_S29_F3454', 'L3_S29_F3457', 'L3_S29_F3460',
       'L3_S29_F3463', 'L3_S29_F3466', 'L3_S29_F3469', 'L3_S29_F3472',
      

#### 코드가 등장한 스테이션을 기준으로 데이터 정리
- 코드 등장한 스테이션 정리:  모든 스테이션에서 코드가 등장하지 않음을 알 수 있음

In [46]:
code_occur_station = []
for idx, row in df_ca.iterrows():
  for col in row.dropna().index:
    if col.split('_')[1] not in code_occur_station:
      code_occur_station.append(col.split('_')[1])

In [47]:
code_occur_station

['S29',
 'S24',
 'S26',
 'S25',
 'S27',
 'S44',
 'S47',
 'S32',
 'S43',
 'S28',
 'S22',
 'S10',
 'S21',
 'S9',
 'S16',
 'S1',
 'S4',
 'S2']

#### id별 등장한 코드와 스테이션 정리

- 코드가 스테이션에 등장했으면 1, 아니면 0을 부여

In [48]:
row = df_ca.iloc[50]
row.dropna().reset_index() # 데이터 프레임으로 만들기

,index,104
0,L2_S26_F3038,T1
1,L2_S26_F3042,T1
2,L2_S26_F3045,T1
3,L2_S26_F3049,T1
4,L2_S26_F3053,T1
...,...,...
85,L3_S29_F3481,T1
86,L3_S29_F3484,T1
87,L3_S29_F3487,T1
88,L3_S29_F3490,T1


In [49]:
# 스테이션이 등장한 경우 (전체행 index 컬럼에서 S26 등장한 경우 뽑기)
row.dropna().reset_index().iloc[:,0].str.contains('S26')

0      True
1      True
2      True
3      True
4      True
      ...  
85    False
86    False
87    False
88    False
89    False
Name: index, Length: 90, dtype: bool

In [50]:
# 코드가 등장한 경우 (전체행, 104컬럼에서 T1코드가 등장한 경우 뽑기)
row.dropna().reset_index().iloc[:,1].str.contains('T1')

0     True
1     True
2     True
3     True
4     True
      ... 
85    True
86    True
87    True
88    True
89    True
Name: 104, Length: 90, dtype: bool

In [51]:
import itertools
from itertools import product
# AND 조건으로 묶어서 데이터 생성
code_x_columns = [] # 스테이션_코드 컬럼 생성
for station, code in itertools.product(code_occur_station, codes): # station과 code의 모든 조합 생성
  code_x_columns.append(station + ' ' +code)

In [52]:
len(code_x_columns)

522

- 모든 행을 iterrows()로 순회하며, 해당 행에 코드가 등장했었던 스테이션과 코드를 조합해서 행별로 True/False 저장 -> int화 해서 1과 0으로 데이터 생성

In [ ]:
# 모든 Id(행)별 등장한 코드와 스테이션을 탐색하기에 시간이 오래 걸린다.
code_x = []

for idx, row in df_ca.iterrows():
    if sum(row.notnull()) == 0: # 모든 컬럼이 결측치인 경우
        record = [0] * len(code_occur_station) * len(codes) # 스테이션과 코드의 조합 가짓수 길이를 0으로

    else: # 컬럼 중 코드 값이 있는 경우
        record = []

        for station, code in itertools.product(code_occur_station, codes): # 스테이션과 코드의 조합
            drop_row = row.dropna().reset_index()
            condition = (drop_row.iloc[:, 0].str.contains(station)) & (drop_row.iloc[:, 1].str.contains(code))
            record.append(sum(condition) > 0) # True/False

    code_x.append(record)

code_x = pd.DataFrame(code_x, columns=code_x_columns, index=df_ca.index)
code_x = code_x.astype(int)

In [54]:
len(code_x)

9488

In [55]:
len(df_ca.index)

10000

데이터 저장

In [ ]:
# 데이터 merge
X = pd.merge(numeric_x, code_x, left_index=True, right_index=True)

# 데이터 저장
X.to_csv('data/data_x.csv')

### 모델링

- 거의 모든 데이트들이 이진형(0과 1)이다
- 샘플과 특징이 모두 많으므로 서포트 백터 머신(SVC) 사용
- 클래스 불균형은 class_weight로 조정

#### X data read, Y data load

In [56]:
data_x = pd.read_csv('data/data_x.csv', index_col=0)

In [57]:
X = data_x.copy()
X.head()

,S0,S1,S2,S3,S4,S5,S6,S7,S8,S9,...,S2 T6,S2 T7,S2 T-2147483648,S2 T48576,S2 T48,S2 T9,S2 T-21474819,S2 T97,S2 T32,S2 T32896
Id,,,,,,,,,,,,,,,,,,,,,
4,1,1,1,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,1,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,1,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
11,1,1,0,1,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
numeric = pd.read_csv('data/train_numeric.csv.zip', nrows=10000)
df = numeric.copy()

df.set_index('Id', inplace=True)

Y = df['Response']

### 모델 학습

In [59]:
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import *

- 학습, 평가 데이터 분리

In [60]:
train_x, test_x, train_y, test_y = train_test_split(X, Y, stratify=Y)

train_x.shape, train_y.shape, test_x.shape, test_y.shape

((7500, 578), (7500,), (2500, 578), (2500,))

In [61]:
train_x.head()

,S0,S1,S2,S3,S4,S5,S6,S7,S8,S9,...,S2 T6,S2 T7,S2 T-2147483648,S2 T48576,S2 T48,S2 T9,S2 T-21474819,S2 T97,S2 T32,S2 T32896
Id,,,,,,,,,,,,,,,,,,,,,
5785,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17909,1,1,1,0,0,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
15316,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19896,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13962,1,1,0,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


MINMAX 스케일링 진행

In [62]:
scaler = MinMaxScaler().fit(train_x)

train_x = pd.DataFrame(scaler.transform(train_x), columns=train_x.columns, index=train_x.index)
test_x = pd.DataFrame(scaler.transform(test_x), columns=test_x.columns, index=test_x.index)

In [63]:
train_x.head()

,S0,S1,S2,S3,S4,S5,S6,S7,S8,S9,...,S2 T6,S2 T7,S2 T-2147483648,S2 T48576,S2 T48,S2 T9,S2 T-21474819,S2 T97,S2 T32,S2 T32896
Id,,,,,,,,,,,,,,,,,,,,,
5785,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17909,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19896,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13962,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


하이퍼 파라미터 설정
- class_weight을 조정  
서포트 백터 머신(SVM)에 적용되는 커널은 대부분 기저함수로 변환하였을 때 무한대의 차원을 가지기에 비선형성을 처리할 수 있다.
서포트 백터 머신의 대표 커널인 linear(선형) 커널과 RBF(가우시안) 커널의 경우로 나눠서 사용하기로 한다.
그리고 하이퍼 파라미터의 수가 많기에 조절하면서 진행하도록 한다.

In [64]:
# CI = 클래스 불균형 비율
CI = train_y.value_counts().iloc[0] / train_y.value_counts().iloc[-1]
CI > 186.5

False

In [65]:
# linear 커널
param_grid_linear = ParameterGrid({
                                    'C':[10**-2, 10**-1, 10**0, 10**1, 10**2],
                                    'class_weight':[{0:1, 1:CI * w} for w in np.arange(0.1, 1.1, 0.2)],
                                    'kernel':['linear'],
                                    'random_state':[29, 1000]
})
# 가우시안 커널
param_grid_rbf = ParameterGrid({
                                    'C':[10**-2, 10**-1, 10**0, 10**1, 10**2],
                                    'class_weight':[{0:1, 1:CI * w} for w in np.arange(0.1, 1.1, 0.2)],
                                    'kernel':['rbf'],
                                    'random_state':[29, 1000],
                                    'gamma':[10**-2, 10**-1, 10**0, 10**1, 10**2]
})

In [66]:
# param_grid_linear = ParameterGrid({
#                                     'C':[10**-2, 10**0, 10**2],
#                                     'class_weight':[{0:1, 1:CI * w} for w in np.arange(0.1, 1.1, 0.2)],
#                                     'kernel':ㅁ['linear'],
#                                     'random_state':[29, 1000]
# })

# param_grid_rbf = ParameterGrid({
#                                     'C':[10**-2, 10**0, 10**2],
#                                     'class_weight':[{0:1, 1:CI * w} for w in np.arange(0.1, 1.1, 0.2)],
#                                     'kernel':['rbf'],
#                                     'random_state':[29, 1000],
#                                     'gamma':[10**-2, 10**0, 10**2]
# })

### 모델 학습

In [67]:
train_x.shape

(7500, 578)

In [68]:
max_iter = 0

for k in range(150, 10, -10):
    for grid in [param_grid_linear, param_grid_rbf]:
        for param in grid:
            max_iter += 1

print(max_iter)

4200


In [69]:
best_score = -1
iter_num = 0

for k in range(150, 10, -10):
    print(k)
    selector = SelectKBest(mutual_info_classif, k=k).fit(train_x, train_y)
    selected_features = train_x.columns[selector.get_support()]

    for grid in [param_grid_linear, param_grid_rbf]:
        for param in grid:
            model = SVC(**param).fit(train_x[selected_features], train_y)
            pred = model.predict(test_x[selected_features])
            score = f1_score(test_y, pred)

            if score > best_score:
                best_score = score
                best_model = model
                best_features = selected_features

            iter_num += 1
            print(f'{iter_num}/{max_iter} best score : {best_score}')

150
1/4200 best score : 0.0
2/4200 best score : 0.0
3/4200 best score : 0.07142857142857144
4/4200 best score : 0.07142857142857144
5/4200 best score : 0.07142857142857144
6/4200 best score : 0.07142857142857144
7/4200 best score : 0.07142857142857144
8/4200 best score : 0.07142857142857144
9/4200 best score : 0.07142857142857144
10/4200 best score : 0.07142857142857144
11/4200 best score : 0.07142857142857144
12/4200 best score : 0.07142857142857144
13/4200 best score : 0.07142857142857144
14/4200 best score : 0.07142857142857144
15/4200 best score : 0.07142857142857144
16/4200 best score : 0.07142857142857144
17/4200 best score : 0.07142857142857144
18/4200 best score : 0.07142857142857144
19/4200 best score : 0.07142857142857144
20/4200 best score : 0.07142857142857144
21/4200 best score : 0.07142857142857144
22/4200 best score : 0.07142857142857144
23/4200 best score : 0.07142857142857144
24/4200 best score : 0.07142857142857144
25/4200 best score : 0.07142857142857144
26/4200 best

KeyboardInterrupt: 

In [70]:
best_score

0.11764705882352941

In [71]:
best_features

Index(['S0', 'S1', 'S3', 'S7', 'S12', 'S13', 'S16', 'S26', 'S29', 'S30',
       ...
       'S4 T3', 'S4 T65536', 'S4 T16777232', 'S4 T8', 'S4 T-2147482816',
       'S2 T145', 'S2 T65536', 'S2 T128', 'S2 T1310', 'S2 T6'],
      dtype='object', length=130)

### 예측 진행
- pipeline 함수
 - 새로운 데이터(2개의 데이터:numeric, categorical)에 대한 예측 수행을 위해 하나의 함수 형태로 파이

In [72]:
# TEST 데이터 불러오기
numeric_test_df = pd.read_csv('data/test_numeric.csv.zip', nrows=10000)
categorical_test_df = pd.read_csv('data/test_categorical.csv.zip', nrows=10000)

In [73]:
def pipeline(numeric_df, categorical_df,
             total_stations, remove_outliers, extract_statistical_feature,
             codes, code_occur_station,
             scaler, model, features):
    # 데이터 카피
    numeric_df_copy = numeric_df.copy()
    categorical_df_copy = categorical_df.copy()

    ## 수치형 데이터 정제
    numeric_df_copy.set_index('Id', inplace = True)

    # station_X 생성
    station_X = []

    for ind, row in numeric_df_copy.iterrows():
        if sum(row.notnull()) == 0:
            station_X.append(np.zeros(len(total_stations))) # whole stations에 포함된 stations를 추가
        else:
            not_null_columns = numeric_df_copy.columns[row.notnull()]
            stations = pd.Series(not_null_columns).str.split('_', expand = True).iloc[:, 1].drop_duplicates().tolist()

            station_X.append(np.isin(total_stations, stations)) # whole stations에 포함된 stations를 추가

    station_X = pd.DataFrame(station_X, index = numeric_df_copy.index, columns = total_stations)
    station_X = station_X.astype(int)

    # stat_feature_X 생성
    stat_feature_X = numeric_df_copy.apply(extract_statistical_feature, axis = 1)
    stat_feature_X.rename({0:"mean", 1: "variance", 2:"max", 3:"min", 4:"kurtosis", 5:"RMS"}, axis = 1, inplace = True)

    numeric_X = pd.merge(station_X, stat_feature_X, left_index = True, right_index = True)

    ## 범주형 데이터 정제
    categorical_df_copy.set_index('Id', inplace = True)

    # code_X_columns 생성
    code_X_columns = []
    for station, code in itertools.product(code_occur_station, codes):
        code_X_columns.append(station + "_" + code)

    # code_X 생성
    code_X = []
    for ind, row in categorical_df_copy.iterrows():
        if sum(row.notnull()) == 0: # 전부 결측이면 굳이 아래 for문을 순회할 필요가 없음
            record = [0] * len(code_occur_station) * len(codes)

        else:
            record = []
            for station, code in itertools.product(code_occur_station, codes):
                dropna_row = row.dropna().reset_index()
                cond = (dropna_row.iloc[:, 0].str.contains(station)) & (dropna_row.iloc[:, 1].str.contains(code))
                record.append(sum(cond) > 0) # True, False로 값을 넣음
        code_X.append(record)

    code_X = pd.DataFrame(code_X, columns = code_X_columns, index = categorical_df_copy.index)
    code_X = code_X.astype(int)

    # X 생성
    X = pd.merge(numeric_X, code_X, left_index = True, right_index = True)

    # 스케일링
    X = pd.DataFrame(scaler.transform(X), columns = X.columns, index = X.index)

    # 특징 선택
    X = X[features]

    # 모델 예측
    output = model.predict(X)

    return output

In [ ]:
output = pipeline(numeric_test_df, categorical_test_df,
             whole_station, remove_outliers, extract_statistical_feature,
             codes, code_occur_station,
             scaler, best_model, best_features)

In [ ]:
# 저장
output.to_csv('data/submission.csv')